In [11]:
import pandas as pd
import os

In [14]:
cwd = os.getcwd()
print(cwd)

/home/guido/code/data-Zee/MMM-project-lewagon/api


In [33]:
parent_dir = '/home/guido/code/data-Zee/MMM-project-lewagon'
data_dir = 'raw_data'
data_file = 'df_clean.csv'

In [34]:
data_path = os.path.join(parent_dir, data_dir, data_file)
print(data_path)

/home/guido/code/data-Zee/MMM-project-lewagon/raw_data/df_clean.csv


In [35]:
data_path

'/home/guido/code/data-Zee/MMM-project-lewagon/raw_data/df_clean.csv'

In [36]:
df = pd.read_csv(data_path)

In [4]:
df = df.drop(columns='Unnamed: 0')
# df['Day'] = pd.to_datetime(df['Day'])
# df.set_index('Day', inplace=True)
# df = df.rename(columns={"fb_costs": "facebook", "google_costs": "google", "tt_costs": "tiktok"})

# # create a simple dataframe

# simple_df = df.drop(columns = ['fb_impressions', 'fb_clicks', 'google_impressions', 'google_clicks', 'tt_impressions', 'tt_clicks'])

# X_simple = simple_df.drop(columns = ['orders', 'total_sales']) # 2 channels: facebook and google
# y_simple = simple_df['total_sales'] # for now lets just use total_sales and exclude orders

In [38]:
simple_df

,orders,total_sales,facebook,google,tiktok
Day,,,,,
2021-07-01,10,414.85,257.01,1.17,0.0
2021-07-02,6,229.95,250.30,0.84,0.0
2021-07-03,10,459.89,248.59,0.72,0.0
2021-07-04,17,508.36,258.05,0.07,0.0
2021-07-05,13,399.88,257.47,0.38,0.0
...,...,...,...,...,...
2023-08-20,294,13616.97,2148.53,466.56,0.0
2023-08-21,356,15348.42,2210.01,400.58,0.0
2023-08-22,150,6054.53,1335.84,289.46,0.0


In [7]:
from mamimo.carryover import ExponentialCarryover
from mamimo.saturation import ExponentialSaturation
from mamimo.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [9]:
# initiate model
adstock = ColumnTransformer(
    [
     ('facebook_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', ExponentialSaturation())
     ]), ['facebook']),
     ('google_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', ExponentialSaturation())
     ]), ['google']),
     ('tiktok_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', ExponentialSaturation())
     ]), ['tiktok'])
])

model = Pipeline([
    ('adstock', adstock),
    ('regression', LinearRegression(positive=True))
])

In [10]:
model

Pipeline(steps=[('adstock',
                 ColumnTransformer(transformers=[('facebook_pipe',
                                                  Pipeline(steps=[('carryover',
                                                                   ExponentialCarryover()),
                                                                  ('saturation',
                                                                   ExponentialSaturation())]),
                                                  ['facebook']),
                                                 ('google_pipe',
                                                  Pipeline(steps=[('carryover',
                                                                   ExponentialCarryover()),
                                                                  ('saturation',
                                                                   ExponentialSaturation())]),
                                                  ['google']),
                                                 ('tiktok_pipe',
                                                  Pipeline(steps=[('carryover',
                                                                   ExponentialCarryover()),
                                                                  ('saturation',
                                                                   ExponentialSaturation())]),
                                                  ['tiktok'])])),
                ('regression', LinearRegression(positive=True))])

In [39]:
model.predict(simple_df)

NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.